In [3]:
import os
import json
import torch
import torch.optim as optim
import torch.nn as nn
import pandas as pd
import numpy as np
import optuna
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [4]:
### DATA INGESTION

class SensorDataset(Dataset):
    def __init__(self, csv_dir, labels_dict, statistical_processing=False):
        self.csv_dir = csv_dir
        self.labels_dict = labels_dict
        self.file_list = [f for f in os.listdir(csv_dir) if f in self.labels_dict]
        self.statistical_processing = statistical_processing

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_name = self.file_list[idx]
        file_path = os.path.join(self.csv_dir, file_name)
        
        df = pd.read_csv(file_path)

        features = df.iloc[:, 1:].values
        if self.statistical_processing:
            # Process for each feature -> x3
            mean_cols = np.tile(np.mean(features, axis=0), (25, 1))
            std_cols = np.tile(np.std(features, axis=0), (25, 1))
            features = np.hstack([features, mean_cols, std_cols])
            
        features = torch.tensor(features, dtype=torch.float32)
        label = torch.tensor(self.labels_dict[file_name], dtype=torch.float32)

        return features, label

def create_dataloader(dataset, csv_dir, labels_dict, statistical_processing=False, batch_size=4):
    data = dataset(csv_dir, labels_dict, statistical_processing)
    return DataLoader(data, batch_size=batch_size, shuffle=True)

In [5]:
### MODELS

class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# 1D CNN -> comparable to RNN with less overhead
class CNN1DClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(CNN1DClassifier, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv1d(in_channels=input_size, out_channels=32, kernel_size=3),
            nn.ReLU(),
            nn.Conv1d(32, 64, kernel_size=3),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        # x = x.transpose(1, 2)
        x = self.conv_block(x)
        x = x.squeeze(-1)
        return self.fc(x)
    
# 3-layer MLP -> worst
class MLPClassifier(nn.Module):
    def __init__(self, input_size, num_classes, dropout_rate=0.2):
        super(MLPClassifier, self).__init__()
        self.network = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_size, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            
            nn.Linear(128, 64),
            nn.ReLU(),
            
            nn.Linear(64, 32),
            nn.ReLU(),
            
            nn.Linear(32, num_classes)
        )

    def forward(self, x):
        return self.network(x)

In [12]:
### TRAINING

def train_model(dataloader, model, criterion, optimiser, epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        
        for batch_features, batch_labels in dataloader:
            if isinstance(model, CNN1DClassifier):
                batch_features = batch_features.transpose(1, 2)
            batch_features = batch_features.to(device)
            batch_labels = batch_labels.to(device).long()
            
            optimiser.zero_grad()
            outputs = model(batch_features)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimiser.step()
            
            total_loss += loss.item()
            
        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")

    # return loss.item()         # For Optuna, eventually

statistical_processing = False
data_folder_path = "data/dummy/dataset"
with open("data/dummy/dataset/labels.json", "r") as f:
    labels_dict = json.load(f)
dataloader = create_dataloader(SensorDataset, data_folder_path, labels_dict, statistical_processing, batch_size=5)

# model = LSTMClassifier(input_size=8*3 if statistical_processing else 8, hidden_size=64, num_layers=2, num_classes=8)
model = CNN1DClassifier(input_size=8, num_classes=8)
# model = MLPClassifier(input_size=8, num_classes=8)

lr = 0.001
batch_size = 16
epochs = 20
criterion = nn.CrossEntropyLoss()
optimiser = optim.Adam(model.parameters(), lr)

train_model(dataloader, model, criterion, optimiser, epochs)

Epoch 1/20 - Loss: 20.2222
Epoch 2/20 - Loss: 12.8832
Epoch 3/20 - Loss: 11.7714
Epoch 4/20 - Loss: 10.5238
Epoch 5/20 - Loss: 9.0120
Epoch 6/20 - Loss: 7.1293
Epoch 7/20 - Loss: 5.2832
Epoch 8/20 - Loss: 4.7998
Epoch 9/20 - Loss: 4.2953
Epoch 10/20 - Loss: 2.4894
Epoch 11/20 - Loss: 1.4772
Epoch 12/20 - Loss: 3.0045
Epoch 13/20 - Loss: 2.9639
Epoch 14/20 - Loss: 2.2237
Epoch 15/20 - Loss: 2.3901
Epoch 16/20 - Loss: 2.5312
Epoch 17/20 - Loss: 2.5129
Epoch 18/20 - Loss: 2.4286
Epoch 19/20 - Loss: 1.9583
Epoch 20/20 - Loss: 1.3568


In [ ]:
### PREDICTION AND EVALUATION

def get_features_from_csv(file_path, statistical_processing=False):
    df = pd.read_csv(file_path)
    features = df.iloc[:, 1:].values

    if statistical_processing:
        mean_cols = np.tile(np.mean(features, axis=0), (25, 1))
        std_cols = np.tile(np.std(features, axis=0), (25, 1))
        features = np.hstack([features, mean_cols, std_cols])

    return features

def predict_csv(model, features):
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if isinstance(model, CNN1DClassifier):      # TODO: test if moving it down here works
        features = features.transpose()
    input_tensor = torch.tensor(features, dtype=torch.float32).unsqueeze(0).to(device)
    
    with torch.no_grad():
        logits = model(input_tensor)
        probs = torch.softmax(logits, dim=1)
        
        pred_class = torch.argmax(probs, dim=1).item()
        confidence = torch.max(probs).item()
        
    return pred_class, confidence

def evaluate_folder(model, folder_path, labels_dict, statistical_processing=False):       
    y_true = []
    y_pred = []

    for file_name, label in labels_dict.items():
        file_path = os.path.join(folder_path, file_name)
        features = get_features_from_csv(file_path, statistical_processing)

        pred, confidence = predict_csv(model, features)
        y_true.append(label)
        y_pred.append(pred)

    accuracy = accuracy_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred, labels=list(range(8)))
    report = classification_report(y_true, y_pred, labels=list(range(8)), zero_division=0)
    
    return accuracy, conf_matrix, report

acc, cm, report = evaluate_folder(model, data_folder_path, labels_dict, statistical_processing)

### Exploration with Kaggle data

Aim is to validate model choice, and that entire pipeline from data ingestion to prediction works.

Data from: https://www.kaggle.com/datasets/harrisonlou/imu-glove/data <br>
Note that data files are not standardised, so I had to do windowing.

#### Results

| Model | Accuracy / % |
|---|---|
| RNN + LSTM (with mean, std dev) | 100 |
| RNN + LSTM | 100 |
| 1D CNN | 99.2 |
| MLP| 99.0 |

In [19]:
class WindowedCSVDataset(Dataset):
    def __init__(self, csv_dir, labels_dict, statistical_processing=False, window_size=25, stride=5):
        self.samples = []
      
        for file_name, label in labels_dict.items():
            file_path = os.path.join(csv_dir, file_name)
            if not os.path.exists(file_path):
                continue

            df = pd.read_csv(file_path)
            x = torch.tensor(df.iloc[:, 1:].values, dtype=torch.float32)  # [T, F]

            T, F = x.shape

            # handle short CSVs
            if T < window_size:
                pad = torch.zeros(window_size - T, F)
                window = torch.cat([x, pad], dim=0)  # [window_size, F]
                self.samples.append((window, torch.tensor(label, dtype=torch.long)))

            for start in range(0, T - window_size + 1, stride):
                window = x[start:start + window_size] # [25, 44]
                
                if statistical_processing:
                    # Calculate stats for this specific window
                    w_mean = window.mean(dim=0) # [44]
                    w_std = window.std(dim=0)   # [44]
                    
                    # Expand stats to match window length [25, 44]
                    mean_feat = w_mean.unsqueeze(0).expand(window_size, -1)
                    std_feat = w_std.unsqueeze(0).expand(window_size, -1)
                    
                    # Concat: [25, 44 + 44 + 44] -> [25, 132]
                    window = torch.cat([window, mean_feat, std_feat], dim=-1)
                
                self.samples.append((window, torch.tensor(label, dtype=torch.long)))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

In [21]:
imu_data_path = "data/kaggle_imu/data"
imu_label_path = "data/kaggle_imu/label"        # labels are [0, 1, 2, 3, 4, 5, 6, 10->7]

def get_labels(label_folder_path):
    labels = dict()

    for file_name in os.listdir(label_folder_path):
        file_path = os.path.join(label_folder_path, file_name)
        data = pd.read_csv(file_path)
        label = data.iat[0,0]
        file_name = file_name.replace("label", "data")
        labels[file_name] = label if label != 10 else 7

    return labels

labels_kaggle = get_labels(imu_label_path)

In [22]:
### TRAINING

statistical_processing = False
dataloader = create_dataloader(WindowedCSVDataset, imu_data_path, labels_kaggle, statistical_processing, batch_size=5)

# model_kaggle = LSTMClassifier(input_size=44*3 if statistical_processing else 44, hidden_size=64, num_layers=2, num_classes=8)
# model_kaggle = MLPClassifier(input_size=25*44, num_classes=8)     # window_size * features
model_kaggle = CNN1DClassifier(input_size=44, num_classes=8)

lr = 0.001
batch_size = 16
epochs = 20
criterion = nn.CrossEntropyLoss()
optimiser = optim.Adam(model_kaggle.parameters(), lr)

train_model(dataloader, model_kaggle, criterion, optimiser, epochs)

Epoch 1/20 - Loss: 0.4571
Epoch 2/20 - Loss: 0.1809
Epoch 3/20 - Loss: 0.1083
Epoch 4/20 - Loss: 0.0928
Epoch 5/20 - Loss: 0.0681
Epoch 6/20 - Loss: 0.0601
Epoch 7/20 - Loss: 0.0533
Epoch 8/20 - Loss: 0.0470
Epoch 9/20 - Loss: 0.0469
Epoch 10/20 - Loss: 0.0380
Epoch 11/20 - Loss: 0.0425
Epoch 12/20 - Loss: 0.0367
Epoch 13/20 - Loss: 0.0393
Epoch 14/20 - Loss: 0.0338
Epoch 15/20 - Loss: 0.0318
Epoch 16/20 - Loss: 0.0305
Epoch 17/20 - Loss: 0.0329
Epoch 18/20 - Loss: 0.0237
Epoch 19/20 - Loss: 0.0334
Epoch 20/20 - Loss: 0.0295


In [26]:
### PREDICTION AND EVALUATION

def get_windows_from_csv(file_path, window_size=25, stride=5, statistical_processing=False):
    df = pd.read_csv(file_path)
    x = torch.tensor(df.iloc[:, 1:].values, dtype=torch.float32)  # [T, F]
    T, F = x.shape

    if T < window_size:
        pad = torch.zeros(window_size - T, F)
        x = torch.cat([x, pad], dim=0)
        T = window_size

    windows = []
    for start in range(0, T - window_size + 1, stride):
        window = x[start:start + window_size]  # [25, 44]
        
        # Add Stats (Must match the Training Dataset logic exactly!)
        if statistical_processing:
            w_mean = window.mean(dim=0).unsqueeze(0).expand(window_size, -1)
            w_std = window.std(dim=0).unsqueeze(0).expand(window_size, -1)
            window = torch.cat([window, w_mean, w_std], dim=-1) # [25, 132]
            
        windows.append(window)

    return torch.stack(windows)  # [num_windows, 25, 132 or 44]

def predict_csv(model, windows):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    windows = windows.to(device)

    if isinstance(model, CNN1DClassifier):
        windows = windows.transpose(1,2)

    with torch.no_grad():
        logits = model(windows)             # [num_windows, num_classes]
        probs = torch.softmax(logits, dim=1) # [num_windows, num_classes]

        # Aggregate: mean across windows
        avg_probs = probs.mean(dim=0)        # [num_classes]
        pred_class = torch.argmax(avg_probs).item()
        confidence = torch.max(avg_probs).item()

    return pred_class, confidence

def evaluate_folder(model, folder_path, labels_dict, window_size=25, stride=5, statistical_processing=False):
    y_true = []
    y_pred = []

    for file_name, label in labels_dict.items():
        file_path = os.path.join(folder_path, file_name)
        windows = get_windows_from_csv(file_path, window_size, stride, statistical_processing)

        pred, confidence = predict_csv(model, windows)
        y_true.append(label)
        y_pred.append(pred)

    accuracy = accuracy_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred, labels=list(range(8)))
    report = classification_report(y_true, y_pred, labels=list(range(8)), zero_division=0)
    
    return accuracy, conf_matrix, report

acc, cm, report = evaluate_folder(model_kaggle, imu_data_path, labels_kaggle, statistical_processing=statistical_processing)
# RNN + LSTM (with mean, std dev): 1
# RNN + LSTM: 1
# 1D CNN: 0.992
# MLP: 0.990

### Export model

In [ ]:
### VITIS AI ROUTE
"""
Copy wsl_vitis into WSL:
> cp -r /mnt/c/Users/Willson/Desktop/Y4S2/CG4002\ -\ CEG\ Capstone/cg4002_ay2526/ai/wsl_vitis/ .
"""

# .onnx for Vitis AI
def export_model(model):
    model.eval()
    if isinstance(model, CNN1DClassifier):
        dummy_input = torch.randn(1, 44, 25)        # No transpose
    else:
        dummy_input = torch.randn(1, 25, 44)

    torch.onnx.export(model, 
                    dummy_input, 
                    "wsl_vitis/cnn.onnx", 
                    opset_version=13, # Vitis AI likes 13
                    input_names=['input'], 
                    output_names=['output'])

# For expected csv
def get_calibration_data(folder_path, labels_dict, num_samples=100):
    all_samples = []
    
    # Note that we want equal representation of each class, so not random
    # 1 file = 1 sample
    unique_labels = list(set(labels_dict.values()))
    samples_per_class = num_samples // len(unique_labels)
    label_counts = {l: 0 for l in unique_labels}
    
    for file_name, label in labels_dict.items():
        if label_counts[label] >= samples_per_class:
            continue
            
        file_path = os.path.join(folder_path, file_name)
        
        # For 1D-CNN
        features = get_features_from_csv(file_path)
        features = features.transpose()
        
        all_samples.append(features)
        label_counts[label] += 1
        
        if len(all_samples) >= num_samples:
            break

    calib_array = np.array(all_samples).astype(np.float32)
    np.save("wsl_vitis/calibration_data.npy", calib_array)

# For Kaggle data
def get_calibration_data_kaggle(folder_path, labels_dict, num_samples=100):
    all_windows = []
    
    # 1 file = many samples
    files_per_label = max(1, num_samples // (len(labels_dict) * 2)) 
    label_counts = {label: 0 for label in set(labels_dict.values())}
    
    for file_name, label in labels_dict.items():
        if label_counts[label] >= files_per_label:
            continue

        file_path = os.path.join(folder_path, file_name)
        
        # For 1D-CNN
        windows = get_windows_from_csv(file_path)
        windows = windows.transpose(1, 2)
        
        all_windows.append(windows)
        label_counts[label] += 1
        
        if sum(w.shape[0] for w in all_windows) >= num_samples:
            break

    calib_tensor = torch.cat(all_windows, dim=0)[:num_samples]
    np.save("wsl_vitis/calibration_data.npy", calib_tensor.numpy())
    
export_model(model_kaggle)
get_calibration_data_kaggle(imu_data_path, labels_kaggle)

In [ ]:
### VITIS HLS ROUTE